# Robot Interface Example

## C++ Driver

This notebook shows how to connect to the robot and send joint velocity/torque commands to it. Before running the notebook, make sure you enable the FCI interface and run the C++ fr3_bridge as follows:

**Joint Velocity Interface**
```bash
fr3_joint_interface 10.42.0.4 franka velocity
```

**Joint Torque Interface**
```bash
fr3_joint_interface 10.42.0.4 franka torque
```

**Note**: If the robot is connected to another computer, make sure that the two computers are on the same local network. Also use the `tools/multicast_config.py` tool to enable the multicast on both computers:

```bash
sudo tools/multicast_config.py <interface name>
```
where interface name is network interface that is on the same local network with the computer connected to the robot. (The computer running the real-time kernel)

## Python Bridge

Now that the interface is up, we can communicate with the robot through Python. Let's instantiate the communication manager:

In [1]:
import sys 
from FR3Py.robot.interface import FR3Real
robot = FR3Real()

Interface Running...


If the interface the successfully exchanging data between the robot and the Python side, we should be able to read the state of the robot. Otherwise, we will get a None.

In [2]:
robot.getStates()

{'q': array([ 3.19840116e-04, -7.84208998e-01, -2.34529158e-04, -2.35632430e+00,
         4.17956734e-04,  1.57000401e+00,  7.85716726e-01,  0.00000000e+00,
         0.00000000e+00]),
 'dq': array([ 7.97927658e-04, -4.55017990e-04,  2.11986707e-04,  9.29398056e-04,
         2.99767555e-04,  2.20032375e-05, -1.20979616e-04,  0.00000000e+00,
         0.00000000e+00]),
 'T': array([ 0.02708699, -5.1136713 , -0.40705362, 23.02490425,  0.56004238,
         2.61639166, -0.13976884,  0.        ,  0.        ])}

Now that we successfully read the state, let's use a simple Jacobian pseudo inverse controller to command the robot's end-effector along the cartesian Z axis:

In [6]:
import time
import numpy as np
from FR3Py.controllers.jacobianPseudoInv import WaypointController

controller = WaypointController(kp=1.5)
# Read the initial state of the robot
time.sleep(0.2)
state = robot.getStates()
if state is not None:
    print('starting the controller')
    q, dq = state['q'], state['dq']
    p0 = controller.robot.getInfo(q,dq)['P_EE']
    R0 = controller.robot.getInfo(q,dq)['R_EE']
    T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
    
    start_time = time.time()
    while time.time()-start_time < 10:
        time.sleep(0.01)
        t = time.time() - start_time
        x = 0.1 * np.sin(t)
        T= T0 @ np.vstack([np.hstack([np.eye(3), np.array([0,0,x]).reshape(3,1)]), np.array([0,0,0,1])])
        state = robot.getStates()
        q, dq = state['q'], state['dq']
        cmd = controller.compute(q, dq, T_cmd=T)
        robot.sendCommands(cmd)

    robot.sendCommands(np.zeros(9))
    print('Demo ended.')
else:
    print('State is None. check the connection')

starting the controller


## Simple Joystick Control
Now that we can control the pose of the end-effector in the task space, let's command the robot using a joystick. For this example, I use a Logitech Extreme 3D Pro joystick and read it using a helper class based on Pygame:

In [7]:
from pyJoy.joy_daq import JoyManager
import time    
joy1  = JoyManager()
joy1.start_daq(joy_idx=0)
joy1.offset_calibration()
time.sleep(3)

pygame 2.5.1 (SDL 2.28.2, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
Put your stick at zero location and do not touch it


In [9]:
import time
import numpy as np
import datetime
from FR3Py.controllers.jacobianPseudoInv import WaypointController

controller = WaypointController(kp=4)
# Read the initial state of the robot
time.sleep(0.2)
state = robot.getStates()
if state is not None:
    print('starting the controller')
    q, dq = state['q'], state['dq']
    p0 = controller.robot.getInfo(q,dq)['P_EE']
    R0 = controller.robot.getInfo(q,dq)['R_EE']
    T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])

    # Initialize the desired pose
    x0, y0, z0 = 0.0, 0.0, 0.0 
    R0 = np.eye(3)
    start_time = time.time()
    while time.time()-start_time < 10:
        trans = controller.robot.getInfo(q,dq)['P_EE']
        rot = controller.robot.getInfo(q,dq)['R_EE']
        pose = np.vstack([np.hstack([rot, trans.reshape(3,1)]), np.array([0,0,0,1])])
        
        analog, digital = joy1.read_values()
        cmd = np.array([analog[0],analog[1],analog[2]])
        for i in range(3):
            if np.abs(cmd[i])<0.1:
                cmd[i] = 0 

        if digital[0]==0:
            y0 = y0 - cmd[0]*0.2/100
            x0 = x0 - cmd[1]*0.2/100
            z0 = z0 + cmd[2]*0.2/100
        else:
            omega_hat = np.array([[ 0,       -cmd[2],  cmd[1]],
                                [ cmd[2],   0,      -cmd[0]],
                                [-cmd[1],   cmd[0],      0]])
            R0 = R0@(np.eye(3)+omega_hat/100)

        time.sleep(0.01)
        T= T0@np.vstack([np.hstack([R0, np.array([x0,y0,z0]).reshape(3,1)]), np.array([0,0,0,1])])
        state = robot.getStates()
        q, dq = state['q'], state['dq']
        cmd = controller.compute(q,dq, T_cmd=T)
        robot.sendCommands(cmd)
    
    robot.sendCommands(np.zeros(9))
    print('Demo ended.')
else:
    print('State is None. check the connection')

starting the controller
Demo ended.
